<div style="text-align: center; line-height: 0; padding-top: 2px;">
  <img src="https://www.quantiaconsulting.com/logos/quantia_logo_orizz.png" alt="Quantia Consulting" style="width: 600px; height: 250px">
</div>

# Multi Partition Reading (Consumer 2)

## Initialization

Let's initialize a consumer and test reading from multiple partitions

In [ ]:
%load_ext autotime

In [ ]:
from confluent_kafka import Consumer, KafkaError
import json
import qcutils

topic = ''
consumer_group = ''

assert len(topic) > 0, "In order to avoid conflicts during write operation, please name the topic as <surname>-topic"
assert len(consumer_group) > 0, "In order to avoid conflicts during write operation, please name the consumer_group as <surname>-cg"


servers=qcutils.read_config_value("kafka.server") + ":" + str(qcutils.read_config_value("kafka.port"))

In [ ]:
consumerconf = {
        'bootstrap.servers': servers,
        'group.id': consumer_group,
        'auto.offset.reset': 'earliest'
    }

c = Consumer(consumerconf)

c.subscribe([topic])

In [ ]:
waiting = False

try:
    while True:
        msg = c.poll(1.0)                                        # retrieve records
        if msg is None:
            if waiting:
                print(".",end =" ")
            else:
                print("Waiting",end =" ")
                waiting = True
            continue
        elif msg.error():
            print('error: {}'.format(msg.error()))
            waiting = False
        else:
            value = json.loads(msg.value())                      # get the value of the message (json.loads)
            key = msg.key()                                      # get the key of the message
            print("\nConsumed record with key {} and value {} from partition {} "
                  .format(key, value, msg.partition()))
            waiting = False
except KeyboardInterrupt:
    pass 
finally:
    c.close() 

##### ![Quantia Tiny Logo](https://www.quantiaconsulting.com/logos/quantia_logo_tiny.png) 2020 Quantia Consulting, srl. All rights reserved.